In [45]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [46]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [47]:
# read in data
parcels = pd.DataFrame.spatial.from_featureclass(r'E:\REMM\Helper Datasets\Base_Year_2015.gdb\Base_Parcels_2015')
policy_override = pd.read_csv(r".\2015\zoning_parcels_p.csv")

In [48]:
# filter columns
parcels = parcels[['parcel_id_REMM', 'parcel_acre','county_name','SHAPE']].copy()

# exclude utah county
parcels = parcels[parcels['county_name'] != 'UTAH'].copy()

In [49]:
# check tables shape
print('parcels: ',parcels.shape)
print('override: ', policy_override.shape)

parcels:  (595009, 4)
override:  (121203, 14)


In [50]:
# join tables, duplicate polygons are expected
joined = parcels.merge(policy_override, left_on='parcel_id_REMM', right_on='parcel_id', how='inner')
print(joined.shape)

(80046, 18)


In [51]:
# fix weird precision issues
joined['max_dua'] = joined['max_dua'].round(1)
joined['max_far'] = joined['max_far'].round(1)

In [52]:
# export to fc
joined.spatial.to_featureclass(location=os.path.join(gdb,'_00_parcels_with_policy'),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Policy-Layer\\Outputs\\scratch.gdb\\_00_parcels_with_policy'

In [ ]:
# dissolve on "year", "max_dua","max_far", "locnote"
dissolve = arcpy.Dissolve_management(os.path.join(gdb,'_00_parcels_with_policy'), os.path.join(gdb,'_01_dissolved_parcels'),
                          ["year", "max_dua","max_far", 'type1','type2','type3','type4','type5','type6','type7','type8', "locnote"], "", "MULTI_PART")

In [ ]:
# show how many years of staging each grouping has

# copy features and delete identical geometries
dissolve_copy = arcpy.FeatureClassToFeatureClass_conversion(dissolve, gdb, "_02_dissolve_copy")
arcpy.DeleteIdentical_management(dissolve_copy, 'SHAPE', xy_tolerance="1 Meters")

# spatial join the copy dissolved layer to the original dissolved polygons
target_features = dissolve_copy
join_features = dissolve
output_features = os.path.join(gdb, "_03_policy_2015_sj")

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", match_option="WITHIN")